<a href="https://colab.research.google.com/github/SammyDMartin/CC/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Uncomment for Colab
use_cc = False
"""
!pip install "ray[rllib]" torch
!pip install nashpy
!pip install pickle5
!git clone https://github.com/SammyDMartin/CC.git
!cd CC

use_CC = True
"""
import numpy as np

In [2]:
if use_cc == True:
    from CC.cc_search import assess_agent_vs_others,assess_agent_vs_agent,make_selfplay_checkpoint
    from CC.cc_tools import DQNTorchPolicy,DQNTrainer,PPOTorchPolicy,PPOTrainer,A3CTorchPolicy,A3CTrainer,BRToLastPolicy,CoopPolicy,DefectPolicy,RandPolicy,UncoopExploiter
    from CC.cc_tools import IteratedPrisonersDilemma, IteratedChicken,IteratedStagHunt, IteratedBoS, MutalismCoordination, PureCoordination,IteratedAsymBoS
else:
    from cc_search import assess_agent_vs_others,assess_agent_vs_agent,make_selfplay_checkpoint
    from cc_tools import DQNTorchPolicy,DQNTrainer,PPOTorchPolicy,PPOTrainer,A3CTorchPolicy,A3CTrainer,BRToLastPolicy,CoopPolicy,DefectPolicy,RandPolicy,UncoopExploiter
    from cc_tools import IteratedPrisonersDilemma, IteratedChicken,IteratedStagHunt, IteratedBoS, MutalismCoordination, PureCoordination,IteratedAsymBoS

ray version: 1.9.2


In [3]:
opponents = [(DQNTorchPolicy,DQNTrainer),(PPOTorchPolicy,PPOTrainer),(A3CTorchPolicy,A3CTrainer)]
strategies = [BRToLastPolicy]

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from cc_tools import bully, security_vals,maximin,nash_from_RLLIB
from cc_tools import IteratedBoS, IteratedPrisonersDilemma

# Tests that all matrix games are working

In [6]:
games = [IteratedPrisonersDilemma, IteratedChicken,IteratedStagHunt, IteratedBoS, MutalismCoordination, PureCoordination,IteratedAsymBoS]

def pretty_print_matrix(g):
    m_list = g.PAYOUT_MATRIX.tolist()
    print(m_list[0])
    print(m_list[1])


def test_matrix(g):
    print(g.__name__)
    pretty_print_matrix(g)
    #print(nash_from_RLLIB(g))
    print("\nBully action steps (out of pure and 50/50 strats): {}".format(bully(g)))
    print("Maximin action steps (out of pure and 50/50 strats): {}".format(maximin(g)))
    print("Security Vals: {}".format(security_vals(g)))
    print("\n\n")

for g in games:
    test_matrix(g)

IteratedPrisonersDilemma
[[-1, -1], [-3, 0]]
[[0, -3], [-2, -2]]

Bully action steps (out of pure and 50/50 strats): (array([0., 1.]), array([0., 1.]))
Maximin action steps (out of pure and 50/50 strats): (array([0., 1.]), array([0., 1.]))
Security Vals: (-2.0, -2.0)



IteratedChicken
[[0.0, 0.0], [-1.0, 1.0]]
[[1.0, -1.0], [-10.0, -10.0]]

Bully action steps (out of pure and 50/50 strats): (array([0., 1.]), array([0., 1.]))
Maximin action steps (out of pure and 50/50 strats): (array([1., 0.]), array([1., 0.]))
Security Vals: (-1.0, -1.0)



IteratedStagHunt
[[3, 3], [0, 2]]
[[2, 0], [1, 1]]

Bully action steps (out of pure and 50/50 strats): (array([1., 0.]), array([1., 0.]))
Maximin action steps (out of pure and 50/50 strats): (array([0., 1.]), array([0., 1.]))
Security Vals: (1.0, 1.0)



IteratedBoS
[[3.0, 2.0], [0.0, 0.0]]
[[0.0, 0.0], [2.0, 3.0]]

Bully action steps (out of pure and 50/50 strats): (array([1., 0.]), array([0., 1.]))
Maximin action steps (out of pure and 50/50 str

# Tests single Uncooperative agent vs DQN

In [7]:
DQN_c,c_res = make_selfplay_checkpoint(IteratedBoS,(DQNTorchPolicy,DQNTrainer),20)

2022-04-12 17:37:17,203	ERROR syncer.py:111 -- Log sync requires rsync to be installed.
 pid=13728) 2022-04-12 17:37:26,956	INFO dqn.py:142 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
 pid=13728) 2022-04-12 17:37:26,956	INFO trainer.py:745 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
 pid=13728) 2022-04-12 17:37:27,074	WARNING deprecation.py:46 -- DeprecationWarning: `convert_to_non_torch_type` has been deprecated. Use `ray/rllib/utils/numpy.py::convert_to_numpy` instead. This will raise an error in the future!
 pid=13728) 2022-04-12 17:37:27,116	WARNING util.py:57 -- Install gputil for GPU system monitoring.
 pid=13728) 2022-04-12 17:37:27,141	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)`

In [8]:
unrol_df = lambda df : df.trial_dataframes[list(df.trial_dataframes.keys())[0]]

In [9]:
unrol_df(c_res)

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,timesteps_total,timesteps_this_iter,agent_timesteps_total,done,...,info/learner/player_col/learner_stats/mean_q,info/learner/player_col/learner_stats/min_q,info/learner/player_col/learner_stats/max_q,info/learner/player_col/learner_stats/cur_lr,info/learner/player_row/learner_stats/allreduce_latency,info/learner/player_row/learner_stats/grad_gnorm,info/learner/player_row/learner_stats/mean_q,info/learner/player_row/learner_stats/min_q,info/learner/player_row/learner_stats/max_q,info/learner/player_row/learner_stats/cur_lr
0,40.0,10.0,30.500000,10.0,10,0,100,0,200,False,...,0.442447,-0.076176,0.780649,0.0001,0.0,1.026361,0.226202,-0.559922,1.059037,0.0001
1,40.0,10.0,26.833333,10.0,20,0,300,0,600,False,...,0.662093,0.148999,1.086581,0.0001,0.0,0.493304,0.468887,-0.276155,1.109645,0.0001
2,40.0,10.0,26.625000,10.0,10,0,400,0,800,False,...,0.804831,0.274008,1.178355,0.0001,0.0,0.475985,0.607475,-0.157361,1.154390,0.0001
3,40.0,10.0,26.083333,10.0,20,0,600,0,1200,False,...,0.894174,0.481615,1.292190,0.0001,0.0,0.703765,0.768522,0.123397,1.268854,0.0001
4,40.0,10.0,25.125000,10.0,20,0,800,0,1600,False,...,1.005590,0.653579,1.455523,0.0001,0.0,0.469652,0.992629,0.473323,1.430494,0.0001
5,45.0,10.0,25.250000,10.0,20,0,1000,0,2000,False,...,1.258680,0.856405,1.644470,0.0001,0.0,0.279282,1.400297,0.859058,1.604553,0.0001
6,45.0,10.0,24.450000,10.0,10,0,1100,0,2200,False,...,1.388336,0.941209,1.801091,0.0001,0.0,0.321077,1.430378,0.977094,1.712450,0.0001
7,45.0,10.0,24.700000,10.0,10,0,1200,0,2400,False,...,1.562274,1.080451,1.976246,0.0001,0.0,0.325032,1.642005,1.130107,1.843784,0.0001
8,45.0,10.0,25.250000,10.0,20,0,1400,0,2800,False,...,1.839404,1.389061,2.351174,0.0001,0.0,0.289531,1.812568,1.459473,2.157524,0.0001
9,45.0,10.0,25.800000,10.0,20,0,1600,0,3200,False,...,2.165245,1.683337,2.691814,0.0001,0.0,0.266969,2.187988,1.816036,2.540730,0.0001


In [10]:
exanal = assess_agent_vs_agent(policies=[DQNTorchPolicy,UncoopExploiter],checkpoints=[DQN_c],environment=IteratedBoS,iters=5)
unrol_df(exanal)

 pid=10832) 2022-04-12 17:38:04,818	INFO dqn.py:142 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
 pid=10832) 2022-04-12 17:38:04,819	INFO trainer.py:745 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
 pid=10832) 2022-04-12 17:38:04,842	WARNING deprecation.py:46 -- DeprecationWarning: `convert_to_non_torch_type` has been deprecated. Use `ray/rllib/utils/numpy.py::convert_to_numpy` instead. This will raise an error in the future!
 pid=10832) restoring ckpt: not loading objs['filters']
 pid=10832) 2022-04-12 17:38:04,880	WARNING util.py:57 -- Install gputil for GPU system monitoring.
 pid=10832) 2022-04-12 17:38:04,892	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an err

 pid=10832) ray version: 1.9.2
 pid=10832) going to load policy player_row from checkpoint C:\Users\User\ray_results\test\DQN_IteratedBoS_d07a1_00000_0_2022-04-12_17-37-17\checkpoint_000020\checkpoint-20


,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,timesteps_total,timesteps_this_iter,agent_timesteps_total,done,...,timers/learn_time_ms,timers/learn_throughput,info/num_steps_sampled,info/num_agent_steps_sampled,info/num_steps_trained,info/num_agent_steps_trained,info/last_target_update_ts,info/num_target_updates,perf/cpu_util_percent,perf/ram_util_percent
0,0.0,0.0,0.0,10.0,100,0,1000,0,2000,False,...,0.0,0.0,1000,2000,32,64,1000,1,35.916667,59.533333
1,0.0,0.0,0.0,10.0,100,0,2000,0,4000,False,...,0.0,0.0,2000,4000,8032,16064,1504,2,33.200000,59.285714
2,0.0,0.0,0.0,10.0,100,0,3000,0,6000,False,...,0.0,0.0,3000,6000,16032,32064,2512,4,32.885714,58.814286
3,0.0,0.0,0.0,10.0,100,0,4000,0,8000,False,...,0.0,0.0,4000,8000,24032,48064,3520,6,32.528571,58.700000
4,0.0,0.0,0.0,10.0,100,0,5000,0,10000,True,...,0.0,0.0,5000,10000,32032,64064,4528,8,37.512500,59.050000


# Tests DQN vs Uncoop Agent on all games

In [ ]:
strategies = [UncoopExploiter]
#opponents = [(PPOTorchPolicy,PPOTrainer),(DQNTorchPolicy,DQNTrainer),(A3CTorchPolicy,A3CTrainer)]

opponents = [(DQNTorchPolicy,DQNTrainer)]

results_uncoop = assess_agent_vs_others((UncoopExploiter,None),opponents=opponents,strategies=strategies,steps=6)
results_uncoop[0]

In [ ]:
results_DQN = assess_agent_vs_others((DQNTorchPolicy,DQNTrainer),opponents=opponents,strategies=strategies,steps=6)
results_DQN[0]

In [14]:
results_uncoop[0]

,IteratedPrisonersDilemma,IteratedChicken,IteratedStagHunt,IteratedBoS,MutalismCoordination,PureCoordination,IteratedAsymBoS
0,-1.0,-0.55,1.4995,0.506,0.5,0.2530,2.0
1,-1.0,-0.55,1.5000,0.529,0.5,0.2645,2.0
2,-1.0,-0.55,1.5000,0.477,0.5,0.2385,2.0
3,-1.0,-0.55,1.5000,0.514,0.5,0.2570,2.0
4,-1.0,-0.55,1.5000,0.489,0.5,0.2445,2.0
5,-1.0,-0.55,1.5000,0.539,0.5,0.2695,2.0


In [15]:
results_DQN[0]

,IteratedPrisonersDilemma,IteratedChicken,IteratedStagHunt,IteratedBoS,MutalismCoordination,PureCoordination,IteratedAsymBoS
0,-1.25,0.05,0.75,0.25,0.5,0.25,0.5
1,-1.25,0.05,0.75,0.25,0.5,0.25,0.5
2,-1.25,0.05,0.75,0.25,0.5,0.25,0.5
3,-1.25,0.05,0.75,0.25,0.5,0.25,0.5
4,-1.25,0.05,0.75,0.25,0.5,0.25,0.5
5,-1.25,0.05,0.75,0.25,0.5,0.25,0.5


## Normalized results
(note that the DQN did worse than the baseline on everything except for Chicken and Mutualism)

In [16]:
results_DQN[0]-results_uncoop[0]

,IteratedPrisonersDilemma,IteratedChicken,IteratedStagHunt,IteratedBoS,MutalismCoordination,PureCoordination,IteratedAsymBoS
0,-0.25,0.6,-0.7495,-0.256,0.0,-0.0030,-1.5
1,-0.25,0.6,-0.7500,-0.279,0.0,-0.0145,-1.5
2,-0.25,0.6,-0.7500,-0.227,0.0,0.0115,-1.5
3,-0.25,0.6,-0.7500,-0.264,0.0,-0.0070,-1.5
4,-0.25,0.6,-0.7500,-0.239,0.0,0.0055,-1.5
5,-0.25,0.6,-0.7500,-0.289,0.0,-0.0195,-1.5
